# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
import pandas as pd
import sqlite3


In [3]:
conn= sqlite3.connect('data.sqlite')
cur=conn.cursor()

In [4]:
q = ("""SELECT name 
                FROM sqlite_master 
                WHERE type='table';
""")
pd.read_sql(q, conn)

,name
0,productlines
1,offices
2,customers
3,employees
4,orderdetails
5,orders
6,payments
7,products


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [8]:
q=("""SELECT employees.firstName, employees.lastName, offices.city
            FROM employees
            JOIN offices 
            ON employees.officeCode = offices.officeCode
            WHERE offices.city = 'Boston';
""")

pd.read_sql(q, conn)

,firstName,lastName,city
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [16]:
q= ("""
SELECT offices.officecode, offices.city,COUNT(employeeS.employeeNumber)AS num_employees
    FROM offices
    LEFT JOIN employees 
    ON offices.officecode= employees.officecode
    GROUP BY offices.officecode, offices.city
    HAVING num_employees = 0;
    """)

pd.read_sql(q, conn)

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [21]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
q= ("""
SELECT offices.city,COUNT(customers.city) AS num_customers
FROM offices
LEFT JOIN customers
ON offices.city = customers.city
    """)

pd.read_sql(q, conn)

,city,num_customers
0,San Francisco,16


In [23]:
# correction code
q=("""SELECT offices.officeCode, offices.city, COUNT(customers.customerNumber) AS num_customers
JOIN employees ON offices.officeCode = employees.officeCode
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY offices.officeCode, offices.city;
   """)
pd.read_sql(q, conn)

,officeCode,city,num_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [24]:
"""
# How many employees are working in each office
"""
q=("""SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) AS num_employee
FROM offices
JOIN employees ON offices.officeCode = employees.officeCode
GROUP BY offices.officeCode, offices.city;
   """)
pd.read_sql(q, conn)

,officeCode,city,num_employee
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


In [26]:
"""
# Which customers have placed more than 5 orders?
"""

q=("""
SELECT customers.customerName, COUNT(orders.orderNumber) AS num_orders
FROM customers
JOIN orders ON customers.customerNumber = orders.customerNumber
GROUP BY customers.customerName
HAVING num_orders > 5;""")

pd.read_sql(q, conn)


,customerName,num_orders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17


In [27]:
conn.close

<function Connection.close>

In [7]:
"""
What is the total amount of payments made by each customer?
"""

q= ("""
SELECT cus.customerNumber,cus.customerName, COUNT(p.customerNumber) AS num_payments
FROM customers AS cus
JOIN payments as p
ON cus.customerNumber= p.customerNumber  
GROUP BY cus.customerName
""")
pd.read_sql(q, conn)


,customerNumber,customerName,num_payments
0,187,"AV Stores, Co.",3
1,242,Alpha Cognac,3
2,249,Amica Models & Co.,2
3,276,"Anna's Decorations, Ltd",4
4,103,Atelier graphique,3
...,...,...,...
93,298,"Vida Sport, Ltd",2
94,181,Vitachrome Inc.,3
95,144,"Volvo Model Replicas, Co",2
96,475,West Coast Collectables Co.,2


In [13]:
# What are the details of orders that contain products from the 'Motorcycles' product line?
q = ("""
SELECT orders.orderNumber, products.productName, orderdetails.quantityOrdered, orderdetails.priceEach
FROM orders
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
JOIN products ON orderdetails.productCode = products.productCode
JOIN productlines ON products.productLine = productlines.productLine
WHERE productlines.productLine = 'Motorcycles'
GROUP BY products.productName;

""")

pd.read_sql(q, conn)

,orderNumber,productName,quantityOrdered,priceEach
0,10107,1936 Harley Davidson El Knucklehead,29,52.70
1,10108,1957 Vespa GS150,43,52.84
2,10107,1960 BSA Gold Star DBD34,38,73.12
3,10107,1969 Harley Davidson Ultimate Chopper,30,81.35
4,10108,1974 Ducati 350 Mk3 Desmo,31,87.76
5,10108,1982 Ducati 900 Monster,35,64.41
6,10108,1982 Ducati 996 R,27,36.21
7,10107,1996 Moto Guzzi 1100i,39,105.86
8,10107,1997 BMW F650 ST,20,88.90
9,10107,1997 BMW R 1100 S,25,96.92


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [15]:
Q=("""
        SELECT p.productCode, p.productName, e.firstName, e.lastName
        FROM employees e
        JOIN customers as cus
        ON e.employeeNumber = cus.salesRepEmployeeNumber
        JOIN  orders o
        ON  cus.customerNumber = o.customerNumber
        JOIN orderDetails ord
        ON  o.orderNumber = ord.orderNumber
        JOIN  products p
        ON  ord.productCode = p.productCode
        GROUP BY e.firstName, e.LastName, p.productName;
""")
pd.read_sql(q, conn)

,orderNumber,productName,quantityOrdered,priceEach
0,10107,1936 Harley Davidson El Knucklehead,29,52.70
1,10108,1957 Vespa GS150,43,52.84
2,10107,1960 BSA Gold Star DBD34,38,73.12
3,10107,1969 Harley Davidson Ultimate Chopper,30,81.35
4,10108,1974 Ducati 350 Mk3 Desmo,31,87.76
5,10108,1982 Ducati 900 Monster,35,64.41
6,10108,1982 Ducati 996 R,27,36.21
7,10107,1996 Moto Guzzi 1100i,39,105.86
8,10107,1997 BMW F650 ST,20,88.90
9,10107,1997 BMW R 1100 S,25,96.92


In [16]:
# chat GPT code

q = ("""
        SELECT employees.firstName, employees.lastName, products.productName
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
JOIN products ON orderdetails.productCode = products.productCode
GROUP BY employees.firstName, employees.lastName, products.productName;

""")

pd.read_sql(q, conn)

,firstName,lastName,productName
0,Andy,Fixter,18th Century Vintage Horse Carriage
1,Andy,Fixter,1900s Vintage Bi-Plane
2,Andy,Fixter,1900s Vintage Tri-Plane
3,Andy,Fixter,1911 Ford Town Car
4,Andy,Fixter,1913 Ford Model T Speedster
...,...,...,...
1363,Steve,Patterson,The Mayflower
1364,Steve,Patterson,The Queen Mary
1365,Steve,Patterson,The Schooner Bluenose
1366,Steve,Patterson,The Titanic


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [5]:
q=("""
        SELECT p.productCode, p.productName, e.firstName, e.lastName, SUM(ord.quantityOrdered) AS num_products
        FROM employees e
        JOIN customers as cus
        ON e.employeeNumber = cus.salesRepEmployeeNumber
        JOIN  orders o
        ON  cus.customerNumber = o.customerNumber
        JOIN orderDetails ord
        ON  o.orderNumber = ord.orderNumber
        JOIN  products p
        ON  ord.productCode = p.productCode
        GROUP BY e.firstName, e.LastName, p.productName;
""")
pd.read_sql(q, conn)

,productCode,productName,firstName,lastName,num_products
0,S18_3136,18th Century Vintage Horse Carriage,Andy,Fixter,126
1,S24_2841,1900s Vintage Bi-Plane,Andy,Fixter,21
2,S24_4278,1900s Vintage Tri-Plane,Andy,Fixter,77
3,S18_2248,1911 Ford Town Car,Andy,Fixter,78
4,S18_2949,1913 Ford Model T Speedster,Andy,Fixter,231
...,...,...,...,...,...
1363,S700_1938,The Mayflower,Steve,Patterson,63
1364,S700_3962,The Queen Mary,Steve,Patterson,40
1365,S700_1138,The Schooner Bluenose,Steve,Patterson,63
1366,S700_3505,The Titanic,Steve,Patterson,62


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [6]:
q= ("""
SELECT employees.firstName, employees.lastName, COUNT(DISTINCT products.productCode) AS unique_products_sold
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
JOIN products ON orderdetails.productCode = products.productCode
GROUP BY employees.firstName, employees.lastName
HAVING COUNT(DISTINCT products.productCode) > 200;
    """)

pd.read_sql(q, conn)

,firstName,lastName,unique_products_sold


In [4]:
conn.close()

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!